In [19]:
import os
import fnmatch

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from lr_video_face.plots import *

from lir import Xy_to_Xn, metrics

In [20]:

home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')
#db_name = 'colab_main_dataset'
# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'output_3')

#Traducción de nombres
dic_emb = {'Facenet512': 'FaceNet', 'QMagFace':'QMagFace', 'ArcFace':'ArcFace'}
dic_qua = {'tface' : 'SDD-FIQA','ser_fiq': 'SER-FIQ'}

In [21]:

files = []
for root, dir, filenames in os.walk(output_dir):
    for filename in fnmatch.filter(filenames, 'dat*.pd'):
        files.append(os.path.join(root, filename))
   

In [22]:
def generate_plot(file):

    df0 = pd.read_pickle(file)

    if len(pd.unique(df0.Detector)) == 1 and len(pd.unique(df0.Calibrator)) == 1:

        cols = list(pd.unique(df0['Embedding Model']))
        rows = list(pd.unique(df0['Quality Model']))

        #generamos la gráfica con subplots
        fig,ax = plt.subplots(nrows = len(rows), ncols = len(cols), squeeze= False, figsize= (25,10))
        
        for index, df1 in df0.iterrows():

            col = cols.index(df1['Embedding Model'])
            row = rows.index(df1['Quality Model'])

            ax1 = ax[row][col]
            #ax1.ylabel = df1['Embedding Model']
            #ax1.set_title ( f"Quality Model: {dic_qua [df1['Quality Model']]}")
            ax1.set_title ( f"Embedding Model: {dic_emb [df1['Embedding Model']]}",fontsize = 14)

            subplot_new(ax1,df1.Results, df1.Cllr)
            #ax1.set(ylabel= f"Embedding Model: {dic_emb[df1['Embedding Model']]}\n Cllr")
            
            #ax1.set(ylabel= f"Quality Model: {dic_qua[df1['Quality Model']]}\n Cllr")
            ax1.set_ylabel(f"Quality Model: {dic_qua[df1['Quality Model']]}\n Cllr",fontsize = 14)
            ax1.set_xlabel('% of discarded pairs',fontsize=14)
        plt.suptitle('$C_{llr}$ Values',fontsize=18)

        #leyenda global
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, borderaxespad=0.)
        fig.tight_layout()
        #savefig = os.path.join(self.experiments.output_dir, f"cllr_summary_ESX{self.experiments.embedding_model_as_scorer}")
        savefig = os.path.join(os.path.dirname(file), os.path.splitext(file)[0]+'_v4')
        plt.savefig(savefig, dpi=600)
        plt.close() 

In [23]:
def generate_latex(file):
    df0 = pd.read_pickle(file)


    
    

    if len(pd.unique(df0.Detector)) == 1 and len(pd.unique(df0.Calibrator)) == 1:
        df_latex = pd.DataFrame()

        for index, df1 in df0.iterrows():    

            emb_model = df1['Embedding Model']
            qua_model = df1['Quality Model']
            # the results are only received for 2015
            lrs_predicted = df1.Results["lrs_predicted"]
            y_test = df1.Results["y_test"]
            dropouts = df1.Results["quality_drops"]
            common_attributes = df1.Results['common_attributes']
    
            cllr_participants = np.mean(df1.Cllr)

            # imagen promedio (17 comparisons)
            lr_avg = df1.Results["lrs_predicted_2015"]
            y_avg = df1.Results["y_test_2015"]    
    
            cllr_avg = metrics.cllr(np.asarray(lr_avg), np.asarray(y_avg)) 
            df_latex = df_latex.append({'Embedding_model': 'Participants',
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_participants},ignore_index = True)
            df_latex = df_latex.append({'Embedding_model': 'Average Image',
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_avg},ignore_index = True)

            #get cllr per dropout

            
            for d in set(dropouts):
                lr_d = [lr for lr,dropout in zip(lrs_predicted,dropouts) if dropout== d ]
                y_d = [y for y,dropout in zip(y_test,dropouts) if dropout== d ]
                cllr_d = metrics.cllr(np.asarray(lr_d), np.asarray(y_d))

            #solo cambio en el momento de plotear
                df_latex = df_latex.append({'Embedding_model': dic_emb[emb_model],
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_d,
                                             'Dropout': 100*(1-d)},ignore_index = True)

            df_latex.sort_values(by= 'Dropout', inplace = True)
            #df_latex.dropna(inplace= True)

            

            # get cllr per common attributes
            # filter results when dropout = 1 
            lrs = [lr for lr,drop in zip(lrs_predicted,dropouts) if drop== 1] 
            ys = [y for y,drop in zip(y_test,dropouts) if drop== 1]
            comatt = [ca for ca,drop in zip(common_attributes,dropouts) if drop== 1]


            #df_plot2 = pd.DataFrame()
            for c in set(comatt):
                lr_c = [lr for lr,com in zip(lrs,comatt) if com== c ]
                y_c = [y for y,comm in zip(ys,comatt) if comm== c ]
                cllr_c = metrics.cllr(np.asarray(lr_c), np.asarray(y_c))
                df_latex = df_latex.append({'Embedding_model': dic_emb[emb_model],
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_c,
                                             'Common Attributes': c},ignore_index = True)
        
            df_latex.sort_values(by= 'Common Attributes', inplace = True)
            #df_latex.dropna(inplace= True)   

#----
        latex_table = df_latex.to_latex(index=False)
        savelatex = os.path.join(os.path.dirname(file), os.path.splitext(file)[0]+'.tex')

        with open (savelatex,'w') as f:
            f.write(latex_table)

In [24]:
for file in files:
    generate_plot(file)
    generate_latex(file)


0.8441920485052142 [0.21028082 0.43258537 1.05134234]
0.9457964236328882 [0.3061633  0.64424442 1.18042506]
0.9001315379818974 [0.25354414 0.5621419  1.10839568]
1.0169230671429754 [0.39668999 0.7813873  1.31378647]
0.9863318230351357 [0.35220996 0.7324699  1.25499228]
1.0463482371458497 [0.43065556 0.82195701 1.38829173]
1.0845028078880077 [0.46000786 0.85871543 1.47228846]
1.1132810288040424 [0.48584881 0.90313513 1.53892551]
1.1341368512965162 [0.50949543 0.94214375 1.58681265]
1.1548255758070312 [0.52530292 0.98581753 1.63860578]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

1.0364725040533076 [0.24590923 0.59687941 1.4034034 ]
1.0796424413871506 [0.31512865 0.74214428 1.46826422]
1.061549460146057 [0.27536243 0.64852927 1.39797213]
1.161342045947408 [0.38099772 0.88670782 1.66394533]
1.1289790634526147 [0.33940139 0.81779914 1.57073773]
1.1727483748291627 [0.40878842 0.93205718 1.70822379]
1.1784450301923572 [0.419595   0.94860465 1.73182668]
1.196631671375521 [0.43551801 0.96909868 1.76591522]
1.2090875197394628 [0.45586894 0.9878248  1.79816765]
1.2133671861126956 [0.4642875  0.99080868 1.81236066]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

0.6700001446308973 [0.1478135  0.28315314 0.91938612]
0.7353913325830282 [0.21461629 0.4354779  0.94558474]
0.7095720267823069 [0.18437395 0.35987781 0.93224343]
0.7829478954828489 [0.28015009 0.6041093  1.00698624]
0.7586348342158282 [0.24223439 0.51345043 0.9732276 ]
0.8206290492354006 [0.3260119  0.69227964 1.05459988]
0.8526645379585083 [0.37051148 0.75443345 1.09211165]
0.8839434181971184 [0.41781031 0.80588891 1.12335182]
0.905164724570634 [0.4635454  0.83764012 1.1453613 ]
0.9221159269263248 [0.50761644 0.87329293 1.15971867]


...

/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

0.9058281133679578 [0.46526319 0.74528614 1.28343722]
0.952667995805808 [0.32459329 0.62641535 1.27151755]
0.9532030859118779 [0.42330742 0.74528614 1.35945477]
0.9392400794673067 [0.23915404 0.52530716 1.05357188]
0.9566098554601474 [0.31016053 0.61437851 1.09758601]
0.9392400794673067 [0.23915404 0.52530716 1.05357188]
0.9248137293227192 [0.18971225 0.48467573 0.99602296]
0.9248137293227192 [0.18971225 0.48467573 0.99602296]
0.9248137293227192 [0.18971225 0.48467573 0.99602296]
0.9248137293227192 [0.18971225 0.48467573 0.99602296]
1.0864933331811994 [0.47954196 1.4213961  1.71334701]
0.9326612508381285 [0.32278951 0.65018045 1.28839421]
0.9819994273661737 [0.42672188 0.90465566 1.42427196]
0.9227941573149485 [0.27882552 0.62641535 1.27151755]
0.9326612508381285 [0.32278951 0.65018045 1.28839421]
0.9227941573149485 [0.27882552 0.62641535 1.27151755]
0.9182584789314623 [0.210214   0.56218634 1.064947  ]
0.9096114061743038 [0.18618048 0.48430156 0.97391519]
0.9248137293227192 [0.1897122

/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram

0.5093891575729017 [0.32613265 0.46402317 0.63822102]
0.4693485338502054 [0.28090809 0.435534   0.56055847]
0.5015174088117249 [0.31081458 0.46399537 0.62627137]
0.5093891575729017 [0.32613265 0.46402317 0.63822102]
0.5015174088117249 [0.31081458 0.46399537 0.62627137]
0.4914402831909359 [0.29077946 0.43841451 0.61723742]
0.5002656361694935 [0.29998202 0.46405096 0.62036704]
0.5053615741555614 [0.30626862 0.46553425 0.62922354]
0.5053615741555614 [0.30626862 0.46553425 0.62922354]
0.8481387368124351 [0.66871086 0.81432352 1.13251917]
0.8819771635024768 [0.48631413 0.80644906 1.13751859]
0.9017801638727923 [0.58858457 0.81432352 1.15126567]
0.8727938548252402 [0.41312845 0.77506554 0.99707896]
0.8825855385292836 [0.47645083 0.79224103 1.09163001]
0.8727938548252402 [0.41312845 0.77506554 0.99707896]
0.8762068347725578 [0.42604543 0.70983058 0.98086866]
0.8762068347725578 [0.42604543 0.70983058 0.98086866]
0.8762068347725578 [0.42604543 0.70983058 0.98086866]
0.8762068347725578 [0.426045

/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:435: FutureWarning: The fram